In [ ]:
!export CUBLAS_WORKSPACE_CONFIG=:4096:8

In [ ]:
import warnings
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
# import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [ ]:
#!pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import clip
# !pip install git+https://github.com/openai/CLIP.git
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# clip_model, compose = clip.load('ViT-L/14', device = device)
# text_model = text_model.cpu()
# def process(idx_val,arr):
#   if idx_val=='0':
#     arr.append(0)
#   else:
#     arr.append(1)

In [ ]:
import torch

In [ ]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
import torch
import math
from torch import nn
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score,precision_score
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from PIL import Image
from torchvision import transforms
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import distutils.version

In [ ]:
data = pd.read_csv('/DATA/atul_2221cs20/jitendra/MAMI_train_set.csv')  #Drive link where your data is saved
data_test = pd.read_csv('/DATA/atul_2221cs20/jitendra/MAMI_test_set.csv')

In [ ]:
data[:10]

,file_name,misogynous,shaming,stereotype,objectification,violence,Text_Transcription
0,1.jpg,0,0,0,0,0,Milk Milk.zip
1,10.jpg,1,0,0,0,1,ROSES ARE RED
2,1000.jpg,0,0,0,0,0,BREAKING NEWS: Russia releases photo of DONALD...
3,10000.jpg,0,0,0,0,0,MAN SEEKING WOMAN Ignad 18 O
4,10006.jpg,0,0,0,0,0,Me explaining the deep lore of. J.R.R. Tolkein...
5,10007.jpg,0,0,0,0,0,PICTOPHLE APP *Straight white malle starts tal...
6,10008.jpg,0,0,0,0,0,Chinese restaurant faces closure after 30 year...
7,10009.jpg,0,0,0,0,0,YOU WERE THE CHOSEN ONE! YOU WERE MEANT TO DES...
8,1001.jpg,1,0,1,1,1,* 84% 6:07 PM Pull start for dishwasher $19 Li...
9,10010.jpg,1,0,0,1,0,SCIENTISTS HAVE DISCOVERED THAT THERE IS INTEL...


In [ ]:
data_test[:10]

,file_name,misogynous,shaming,stereotype,objectification,violence,Text_Transcription
0,15001.jpg,0,0,0,0,0,G HIS. UNDYING FIDELITY Steve is hot and perfe...
1,15002.jpg,0,0,0,0,0,How limagined myself as a Teacher...... How I ...
2,15004.jpg,0,0,0,0,0,WHERE WILL YOU BE WHEN DIARRHEA STRIKE memecen...
3,15005.jpg,0,0,0,0,0,A MAN WITH DREAMS... NEEDS A WOMAN WITH VISION
4,15006.jpg,0,0,0,0,0,THIS IS HOW YOUR GIRLFRIEND SEES YOUR FEMALE F...
5,15008.jpg,1,0,0,1,1,BITCHES BE LIKE... GOIN TO THE HIGHEST BIDDER ...
6,15009.jpg,0,0,0,0,0,MARRIAGE IS JUST TEXTING EACH OTHER MEMES FROM...
7,15010.jpg,0,0,0,0,0,MARRIAGE IS... ALWAYS HAVING SOMEONE TO PITY-L...
8,15011.jpg,0,0,0,0,0,THE MEDIA: THIS IS A PEACEFUL PROTEST ALSO THE...
9,15012.jpg,0,0,0,0,0,"PARENTING MOMSGOTINK IF YOU FEEL CRAZY, THEN Y..."


In [ ]:
data.head(1)

,file_name,misogynous,shaming,stereotype,objectification,violence,Text_Transcription
0,1.jpg,0,0,0,0,0,Milk Milk.zip


In [ ]:
#!pip install multilingual-clip

In [ ]:
from multilingual_clip import pt_multilingual_clip
import transformers

In [ ]:
import os
#os.environ['CUDA_VISIBLE_DEVICES']='2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# clip_model, compose = clip.load('RN50x4', device = device)
clip_model, compose = clip.load("ViT-B/32", device = device)
text_inputs = (clip.tokenize(data.Text_Transcription.values[321],truncate=True)).to(device)
print(text_inputs)

tensor([[49406,  1139,   531,   886,  1137, 42578, 49407,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], device='cuda:0',
       dtype=torch.int32)


In [ ]:
t_f=clip_model.encode_text(text_inputs)

In [ ]:
#clip_model.encode_text(torch.tensor([[1]*77]).cuda())

In [ ]:
def get_data(data):  # data = pd.read_csv(dataset_path)
    # Extracting lists from the DataFrame
    text = list(data['Text_Transcription'])
    img_path = list(data['file_name'])
    name = list(data['file_name'])
    label = list(data['misogynous'])

    # Initializing lists for storing processed data
    text_features, image_features, l, Name, v, text_descriptions = [], [], [], [], [], []

    for txt, img, L, n in tqdm(zip(text, img_path, label, name)):
        try:
            img = Image.open('/DATA/atul_2221cs20/jitendra/MAMI_2022_images/training_images/' + img)
        except Exception as e:
            print(e)
            continue

        img = torch.stack([compose(img).to(device)])

        l.append(L)
        Name.append(n)
        text_descriptions.append(txt)  # Append the text description to the list

        with torch.no_grad():
            txt = clip.tokenize(txt,truncate=True).to(device)
            temp_txt = clip_model.encode_text(txt).detach().cpu().numpy()

#             temp_txt = model.forward(txt, tokenizer).detach().cpu().numpy()
            text_features.append(temp_txt)

            temp_img = clip_model.encode_image(img).detach().cpu().numpy()
            image_features.append(temp_img)

            del temp_txt
            del temp_img
            torch.cuda.empty_cache()

        del img
        torch.cuda.empty_cache()

    # Return the text descriptions along with the other data
    return text_features, image_features, l, Name, text_descriptions



In [ ]:
#trial
text_features, image_features, l, Name, text_descriptions = get_data(data.head(2))

0it [00:00, ?it/s]

In [ ]:
print(len(text_features))

2


In [ ]:
text_descriptions

['Milk Milk.zip', 'ROSES ARE RED']

In [ ]:
outliers = []
for names in tqdm(list(data['file_name'])):
  #change the path according to your drive
  if not os.path.exists('/DATA/atul_2221cs20/jitendra/MAMI_2022_images/training_images/'+names):
    outliers.append(names)

# data = data[~data['Name'].isin(outliers)]

  0%|          | 0/9941 [00:00<?, ?it/s]

In [ ]:
outliers

[]

In [ ]:
class HatefulDataset(Dataset):
    def __init__(self, data):
        # Assuming get_data now also returns text_descriptions
        self.t_f, self.i_f, self.label, self.name, self.text_descriptions = get_data(data)

        # Squeeze the arrays if necessary (assuming this is for removing unnecessary dimensions)
        self.t_f = np.squeeze(np.asarray(self.t_f), axis=1)
        self.i_f = np.squeeze(np.asarray(self.i_f), axis=1)

    def __len__(self):
        # The length of the dataset is the length of the labels
        return len(self.label)

    def __getitem__(self, idx):
        # Convert idx from tensor to list if it's a tensor
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Extract data for the given index
        name = self.name[idx]
        label = self.label[idx]
        T = self.t_f[idx, :]
        I = self.i_f[idx, :]
        text_description = self.text_descriptions[idx]  # Extract the text description

        # Create a sample dictionary to return
        sample = {
            'label': label,
            'processed_txt': T,
            'processed_img': I,
            'name': name,
            'text_description': text_description  # Include the text description in the sample
        }

        return sample


In [ ]:
#sample_dataset = HatefulDataset(data)

In [ ]:
#sample_dataset[1]

In [ ]:
#len(sample_dataset)

In [ ]:
#torch.save(sample_dataset,'MAMI_train.pt')

In [ ]:
sample_dataset_new= torch.load("MAMI_train.pt")

In [ ]:
#sample_dataset_new[0]

In [ ]:
#test= torch.load("MAMI_train.pt")

In [ ]:
len(sample_dataset_new)

9941

In [ ]:
#loding test data

In [ ]:
def get_data1(data_test):  # data = pd.read_csv(dataset_path)
    # Extracting lists from the DataFrame
    text = list(data_test['Text_Transcription'])
    img_path = list(data_test['file_name'])
    name = list(data_test['file_name'])
    label = list(data_test['misogynous'])

    # Initializing lists for storing processed data
    text_features, image_features, l, Name, v, text_descriptions = [], [], [], [], [], []

    for txt, img, L, n in tqdm(zip(text, img_path, label, name)):
        try:
            img = Image.open('/DATA/atul_2221cs20/jitendra/MAMI_2022_images/test_images/' + img)
        except Exception as e:
            print(e)
            continue

        img = torch.stack([compose(img).to(device)])

        l.append(L)
        Name.append(n)
        text_descriptions.append(txt)  # Append the text description to the list

        with torch.no_grad():
            txt = clip.tokenize(txt,truncate=True).to(device)
            temp_txt = clip_model.encode_text(txt).detach().cpu().numpy()

#             temp_txt = model.forward(txt, tokenizer).detach().cpu().numpy()
            text_features.append(temp_txt)

            temp_img = clip_model.encode_image(img).detach().cpu().numpy()
            image_features.append(temp_img)

            del temp_txt
            del temp_img
            torch.cuda.empty_cache()

        del img
        torch.cuda.empty_cache()

    # Return the text descriptions along with the other data
    return text_features, image_features, l, Name, text_descriptions

In [ ]:
# outliers1 = []
# for names in tqdm(list(data['file_name'])):
#   #change the path according to your drive
#   if not os.path.exists('/DATA/atul_2221cs20/jitendra/MAMI_2022_images/test_images/'+names):
#     outliers1.append(names)

In [ ]:
# outliers1

In [ ]:
class HatefulDataset1(Dataset):
    def __init__(self, data):
        # Assuming get_data now also returns text_descriptions
        self.t_f, self.i_f, self.label, self.name, self.text_descriptions = get_data1(data)

        # Squeeze the arrays if necessary (assuming this is for removing unnecessary dimensions)
        self.t_f = np.squeeze(np.asarray(self.t_f), axis=1)
        self.i_f = np.squeeze(np.asarray(self.i_f), axis=1)

    def __len__(self):
        # The length of the dataset is the length of the labels
        return len(self.label)

    def __getitem__(self, idx):
        # Convert idx from tensor to list if it's a tensor
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Extract data for the given index
        name = self.name[idx]
        label = self.label[idx]
        T = self.t_f[idx, :]
        I = self.i_f[idx, :]
        text_description = self.text_descriptions[idx]  # Extract the text description

        # Create a sample dictionary to return
        sample = {
            'label': label,
            'processed_txt': T,
            'processed_img': I,
            'name': name,
            'text_description': text_description  # Include the text description in the sample
        }

        return sample

In [ ]:
#sample_dataset_t = HatefulDataset1(data_test)

In [ ]:
#torch.save(sample_dataset_t,'MAMI_test.pt')

In [ ]:
sample_dataset_test= torch.load("MAMI_test.pt")

In [ ]:
len(sample_dataset_test)

1000

In [ ]:
sample_dataset_test[1]

{'label': 0,
 'processed_txt': array([-2.7603e-02, -1.2413e-02, -9.5978e-03,  2.7002e-01,  2.6831e-01,
         2.9199e-01, -3.6914e-01, -6.9922e-01, -3.4692e-01, -1.8616e-01,
         2.7734e-01,  2.4304e-01,  3.1714e-01,  1.2321e-02, -6.6345e-02,
        -1.7920e-01, -2.9663e-01, -9.5215e-02, -8.5754e-02, -1.8445e-01,
        -1.0022e-01,  3.8738e-03,  3.1689e-01, -2.7069e-02, -9.8389e-02,
         4.7493e-03,  2.2675e-02, -2.7734e-01, -1.3989e-01,  2.5781e-01,
        -9.4360e-02, -8.2397e-02,  1.8762e-01, -1.3260e-02,  2.7368e-01,
        -3.9520e-02, -5.2094e-02,  6.3049e-02, -1.0437e-01, -6.3916e-01,
         1.6138e-01, -1.7029e-01,  2.6001e-01,  3.1689e-01,  1.7737e-01,
         5.6348e-01, -1.9299e-01, -1.3660e-01, -2.9761e-01, -1.9604e-01,
         1.1432e-01, -2.7246e-01,  1.1139e-01,  2.3657e-01, -2.3389e-01,
         2.4084e-01,  1.1023e-01,  3.5547e-01,  6.8237e-02,  5.8691e-01,
         3.2617e-01,  1.5320e-01,  7.1582e-01, -1.1810e-01, -7.3059e-02,
         2.5940e-03, 

In [ ]:
import pytorch_lightning as pl

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class MFB(nn.Module):  #multi-modal feature fusion between image and question features.

    def __init__(self,img_feat_size, ques_feat_size, is_first, MFB_K, MFB_O, DROPOUT_R):
        super(MFB, self).__init__()  # calls the constructor of the parent class
        #store hyperparameters
        #self.__C = __C
        self.MFB_K = MFB_K #hyperparameter for pooling
        self.MFB_O = MFB_O  #hyperparameter for output dimension
        self.DROPOUT_R = DROPOUT_R
        #print(MFB_K)

        #img_feat_size = 5 (dimension of image feature vectors)

        self.is_first = is_first  #Stores a boolean flag indicating if this is the first MFB module
        self.proj_i = nn.Linear(img_feat_size, MFB_K * MFB_O) #Creates a linear layer for image feature projection

        self.proj_q = nn.Linear(ques_feat_size, MFB_K * MFB_O) #Creates a linear layer for question feature projection

        self.dropout = nn.Dropout(DROPOUT_R) #Creates a dropout layer for regularization
        self.pool = nn.AvgPool1d(MFB_K, stride = MFB_K) #Creates an average pooling layer for dimensionality reduction
        #print(self.pool)
 #self.MFB = MFB(512, 38400, True, 256, 64, 0.1)
    def forward(self, img_feat, ques_feat, exp_in=1):  #Defines how data flows through the module
        '''
            img_feat.size() -> (N, C, img_feat_size)    C = 1 or 100
            ques_feat.size() -> (N, 1, ques_feat_size)
            z.size() -> (N, C, MFB_O)
            exp_out.size() -> (N, C, K*O)
        '''

        batch_size = img_feat.shape[0]
        #print(img_feat.shape)
        #print(img_feat.shape[0])
        img_feat = self.proj_i(img_feat)#Projects image features using self.proj_i (output shape: (N, C, K*O))

        #print(ques_feat.shape)
        #print(img_feat.size())
        ques_feat = self.proj_q(ques_feat)              # (N, 1, K*O)
        #print(ques_feat.size())

       # ques_feat = ques_feat.transpose(0, 1)



        exp_out = img_feat * ques_feat             # (N, C, K*O)
        #print(exp_out)
       # print(exp_out.size())
        exp_out = self.dropout(exp_out) if self.is_first else self.dropout(exp_out * exp_in)     # (N, C, K*O)
        #print(exp_out)
       # print(exp_out.size())
        z = self.pool(exp_out) * self.MFB_K         # (N, C, O)


        z = torch.sqrt(F.relu(z)) - torch.sqrt(F.relu(-z))
        ##print(z)

        z = F.normalize(z.view(batch_size, -1))         # (N, C*O)

        z = z.view(batch_size, -1, self.MFB_O)      # (N, C, O)
        #print(z)
#         print(z.view(batch_size, -1))
#         print(z)
#         print(z.size())
        return z

In [ ]:
#torch.cuda.empty_cache()


In [ ]:
torch.manual_seed(123)
t_p,v_p = torch.utils.data.random_split(sample_dataset_new,[9000,941])

# torch.manual_seed(123)
t_p,te_p1 = torch.utils.data.random_split(t_p,[7000,2000])

te_p=sample_dataset_test

In [ ]:
len(te_p)

1000

In [ ]:
t_p[1]["processed_img"].shape

(512,)

In [ ]:
#  mask_and_predict("hi  how are you")

In [ ]:
#       logit_offen= self.forward(txt,img)
#       pro=torch.sigmoid(logit_offen)
#       print(pro)
#       predicted_labels = torch.argmax(pro, dim=1)
#       print(predicted_labels.tolist )


#       loss = self.cross_entropy_loss(logit_offen, lab)

In [ ]:
print(len(contours))

In [ ]:
#!pip install transformations

In [1]:
#Now we will create a code for applications



In [ ]:
# HERE WE APPLY ATTENTION USING CLIP MODEL AND BLUR REQUIRED REGION

In [ ]:
#@title Install dependencies

#@markdown Please execute this cell by pressing the _Play_ button
#@markdown on the left.

#@markdown **Note**: This installs the software on the Colab
#@markdown notebook in the cloud and not on your computer.

!pip install ftfy regex tqdm matplotlib opencv-python scipy scikit-image
!pip install git+https://github.com/openai/CLIP.git

import urllib.request
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import clip
from PIL import Image
from scipy.ndimage import filters
from torch import nn


In [ ]:
#@title GradCAM: Gradient-weighted Class Activation Mapping

#@markdown Our gradCAM implementation registers a forward hook
#@markdown on the model at the specified layer. This allows us
#@markdown to save the intermediate activations and gradients
#@markdown at that layer.

#@markdown To visualize which parts of the image activate for
#@markdown a given caption, we use the caption as the target
#@markdown label and backprop through the network using the
#@markdown image as the input.
#@markdown In the case of CLIP models with resnet encoders,
#@markdown we save the activation and gradients at the
#@markdown layer before the attention pool, i.e., layer4.
#@title Helper functions

#@markdown Some helper functions for overlaying heatmaps on top
#@markdown of images and visualizing with matplotlib.

def normalize(x: np.ndarray) -> np.ndarray:
    # Normalize to [0, 1].
    x = x - x.min()
    if x.max() > 0:
        x = x / x.max()
    return x

# Modified from: https://github.com/salesforce/ALBEF/blob/main/visualization.ipynb
def getAttMap(img, attn_map, blur=True):
    if blur:
        attn_map = filters.gaussian_filter(attn_map, 0.02*max(img.shape[:2]))
    attn_map = normalize(attn_map)
    cmap = plt.get_cmap('jet')
    attn_map_c = np.delete(cmap(attn_map), 3, 2)
    attn_map = 1*(1-attn_map**0.7).reshape(attn_map.shape + (1,))*img + \
            (attn_map**0.7).reshape(attn_map.shape+(1,)) * attn_map_c
    return attn_map

def viz_attn(img, attn_map, blur=True):
    _, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(img)
    axes[1].imshow(getAttMap(img, attn_map, blur))
    for ax in axes:
        ax.axis("off")
    plt.show()
    p=getAttMap(img, attn_map, blur)
    print('p',p.shape)

    return p

def load_image(img_path, resize=None):
    image = Image.open(img_path).convert("RGB")
    if resize is not None:
        image = image.resize((resize, resize))
    return np.asarray(image).astype(np.float32) / 255.

class Hook:
    """Attaches to a module and records its activations and gradients."""

    def __init__(self, module: nn.Module):
        self.data = None
        self.hook = module.register_forward_hook(self.save_grad)

    def save_grad(self, module, input, output):
        self.data = output
        output.requires_grad_(True)
        output.retain_grad()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.hook.remove()

    @property
    def activation(self) -> torch.Tensor:
        return self.data

    @property
    def gradient(self) -> torch.Tensor:
        return self.data.grad


# Reference: https://arxiv.org/abs/1610.02391
def gradCAM(
    model: nn.Module,
    input: torch.Tensor,
    target: torch.Tensor,
    layer: nn.Module
) -> torch.Tensor:
    # Zero out any gradients at the input.
    if input.grad is not None:
        input.grad.data.zero_()

    # Disable gradient settings.
    requires_grad = {}
    for name, param in model.named_parameters():
        requires_grad[name] = param.requires_grad
        param.requires_grad_(False)

    # Attach a hook to the model at the desired layer.
    assert isinstance(layer, nn.Module)
    with Hook(layer) as hook:
        # Do a forward and backward pass.
        output = model(input)
        output.backward(target)

        grad = hook.gradient.float()
        act = hook.activation.float()

        # Global average pool gradient across spatial dimension
        # to obtain importance weights.
        alpha = grad.mean(dim=(2, 3), keepdim=True)
        # Weighted combination of activation maps over channel
        # dimension.
        gradcam = torch.sum(act * alpha, dim=1, keepdim=True)
        # We only want neurons with positive influence so we
        # clamp any negative ones.
        gradcam = torch.clamp(gradcam, min=0)

    # Resize gradcam to input resolution.
    gradcam = F.interpolate(
        gradcam,
        input.shape[2:],
        mode='bicubic',
        align_corners=False)

    # Restore gradient settings.
    for name, param in model.named_parameters():
        param.requires_grad_(requires_grad[name])

    return gradcam

In [ ]:
#@title Run

#@markdown #### Image & Caption settings
#image_url = '/content/image.png' #@param {type:"string"}

image_caption = 'SHE is hot' #@param {type:"string"}
#@markdown ---
#@markdown #### CLIP model settings
clip_model = "RN50" #@param ["RN50", "RN101", "RN50x4", "RN50x16"]
saliency_layer = "layer4" #@param ["layer4", "layer3", "layer2", "layer1"]
#@markdown ---
#@markdown #### Visualization settings
blur = True #@param {type:"boolean"}

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(clip_model, device=device, jit=False)



In [ ]:
#THIS CODE WOrking properly for bluring using clip RN50 model

In [ ]:
#version05
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import clip
import cv2
import numpy as np
from PIL import Image
import os
import csv
from pytorch_lightning import seed_everything


# Load CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# Define attention mechanism
class SelfAttention(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.query_proj = nn.Linear(feature_dim, feature_dim).to(device)
        self.key_proj = nn.Linear(feature_dim, feature_dim).to(device)
        self.value_proj = nn.Linear(feature_dim, feature_dim).to(device)

    def forward(self, image_features):
        #print(' image_features', image_features.size)
        feature_dim = 512

        # Convert NumPy array to PyTorch tensor and move to device
        image_features = torch.tensor(image_features).to(device).float()
       # print("image_features shape (input):", image_features.shape)


        # Apply linear projections for Q, K, and V
        Q = self.query_proj(image_features)
        K = self.key_proj(image_features)
        V = self.value_proj(image_features)

        # Compute attention scores and weights
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / image_features.shape[-1]**0.5
        attention_weights = F.softmax(attention_scores, dim=-1)


        # Apply attention weights to obtain the weighted features
        weighted_features = torch.matmul(attention_weights, V)


        # Sum along the last dimension to get the final attention output
        #output_features = torch.sum(weighted_features, dim=-2)
        #print(' weighted_features', weighted_features)
        #print("weighted_features shape (output):", weighted_features.shape)

        return weighted_features , attention_weights
# Define PredictionModel
class PredictionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 512).to(device)
        self.text_embedding = nn.Linear(512, 512).to(device)  # Project text to 512 dimensions
        self.output_layer = nn.Linear(512 + 512, num_classes).to(device)  # Combine 1024 features
    def forward(self, masked_features, text_features=None):
        masked_features = masked_features.float()
        x = F.relu(self.fc1(masked_features))

        if text_features is not None:
            text_features = text_features.float()
            text_emb = F.relu(self.text_embedding(text_features))
           # print("x shape:", x.shape)
           # print("text_emb shape:", text_emb.shape)
            x = torch.cat([x, text_emb], dim=1)

        prediction = self.output_layer(x)
        #print('prediction',prediction)
        return prediction

# Define ImageClassifier
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.clip_model = clip_model
        self.attention_layer = SelfAttention(feature_dim=512)
        self.classifier = PredictionModel(input_dim=512, num_classes=2)

    def forward(self, image_tensor, text_tensor):
        #image_tensor = image_tensor.unsqueeze(1)
       # image_tensor = image_tensor.repeat(1, 3, 1, 1)
        #image_features = clip_model.encode_image(image_tensor)
        #print('image_tensor',image_tensor.shape)
        #print('text_tensor',text_tensor.shape)

        masked_features, attention_weights = self.attention_layer(image_tensor)
        #print('masked_features',masked_features)
        #print('masked_features shape',masked_features.shape)
        prediction = self.classifier(masked_features, text_tensor)
        #print('classifier prediction', prediction )
        return prediction, attention_weights
def apply_attention(image, attention_weights, image_path,text_description):

    original_image = cv2.imread(image_path)
    # Download the image from the web.
    image_caption = text_description #@param {type:"string"}
    #@markdown ---
    #@markdown #### CLIP model settings
    clip_model = "RN50" #@param ["RN50", "RN101", "RN50x4", "RN50x16"]
    saliency_layer = "layer4" #@param ["layer4", "layer3", "layer2", "layer1"]
    #@markdown ---
    #@markdown #### Visualization settings
    blur = True #@param {type:"boolean"}

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load(clip_model, device=device, jit=False)
    image_path = image_path
    #urllib.request.urlretrieve(image_url, image_path)

    image_input = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    image_np = load_image(image_path, model.visual.input_resolution)
    text_input = clip.tokenize([image_caption[:77]]).to(device)

   # _, axes = plt.subplots(1, 2, figsize=(10, 5))

    attn_map = gradCAM(
        model.visual,
        image_input,
        model.encode_text(text_input).float(),
        getattr(model.visual, saliency_layer)
    )
    attn_map = attn_map.squeeze().detach().cpu().numpy()
#     with np.printoptions(threshold=np.inf):
#         print(attn_map)
    ##print('attn_map',attn_map.shape)
    binary_mask=(attn_map>0.07)
   # print('binary_mask',binary_mask)

#     plt.figure(figsize=[22,22])
#     plt.subplot(121);plt.imshow(attn_map);plt.title("original Image");plt.axis('off');

    binary_mask_3=np.dstack((binary_mask,binary_mask,binary_mask))

    #print('binary_mask',binary_mask_3.shape)
   # print('attn_map',attn_map.shape)
    blurred_image=cv2.GaussianBlur(image_np,(25, 25),0)
    image= cv2.resize(image, (224,224))


  #  print('blurred_image',blurred_image.shape)
  #  print('image',image.shape)
    blurred_image = (blurred_image * 255).astype('uint8') # Scale and convert to 'uint8'
    blurred_image = cv2.cvtColor(blurred_image, cv2.COLOR_RGB2BGR)

#     axes[0].imshow(blurred_image)
#     axes[1].imshow(image)
#     for ax in axes:
#         ax.axis("off")
#     plt.show()

    o_i=np.where(binary_mask_3,blurred_image,image)
    o_i= cv2.resize(o_i, (640,640))

    #reversed_img = img.transpose(2, 0, 1)
    #plt.imshow(o_i)
#     _, axes = plt.subplots(1, 2, figsize=(10, 5))
#     axes[0].imshow(o_i)
#     axes[1].imshow(getAttMap(image_np, attn_map, blur))
#     for ax in axes:
#         ax.axis("off")
#     plt.show()


   # viz_attn(image_np, attn_map, blur)

    #modified_image = getAttMap(image_np, attn_map, blur)
    #modified_image = (modified_image * 255).astype('uint8') # Scale and convert to 'uint8'
    #modified_image = cv2.cvtColor(modified_image, cv2.COLOR_RGB2BGR)

    #cv2.imwrite("Blurred1eeeee.jpg",  o_i)

    return o_i

def process_image(image_path,attention_weights,t,text_description):
    image = cv2.imread(image_path)


    # Optional: You can use the attention weights for visualization
    attention_heatmap = cv2.resize(attention_weights[0].detach().cpu().numpy(), (image.shape[1], image.shape[0]))

    # Apply blur only to misogynistic regions
    modified_image = apply_attention(image, attention_weights,image_path,text_description)

    # Save the modified image and attention heatmap
    filename, ext = os.path.splitext(os.path.basename(image_path))
    modified_filename = filename + "_modified" + ext
    attention_heatmap_filename = filename + "_attention_heatmap" + ext

    output_dir = "output_images_RN50"+str(t)
    os.makedirs(output_dir, exist_ok=True)

    modified_path = os.path.join(output_dir, modified_filename)
    heatmap_path = os.path.join(output_dir, attention_heatmap_filename)

    cv2.imwrite(modified_path, modified_image)
   # cv2.imwrite(heatmap_path, (attention_heatmap * 255).astype(np.uint8))

    return modified_image, attention_heatmap
# Define Classifier class
class Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.MFB = MFB(512, 512, True, 256, 64, 0.1)
        self.fin_y_shape = torch.nn.Linear(768, 512)
        self.clip_model, self.compose = clip.load("ViT-B/32", device=device)
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
        self.fin_old = torch.nn.Linear(64, 2)  # producing final output logits
        self.fin = torch.nn.Linear(16 * 768, 64)  # linear layer to the fused features
        self.image_classifier = ImageClassifier()

    def mask_and_predict(self, text):
        inputs = clip.tokenize(text, truncate=True).to(self.device)
        text_features = self.clip_model.encode_text(inputs).detach().cpu().numpy()
        text_features = torch.tensor(text_features).to(self.device)
        text_features = text_features.view(text_features.size(0), -1)
        return text_features

    def forward(self, x, y, text):
        x_, y_ = x, y
        text_features = self.mask_and_predict(text)
        z = self.MFB(torch.unsqueeze(y, axis=1), torch.unsqueeze(text_features, axis=1))
        z_new = torch.squeeze(z, dim=1)
        c = self.fin_old(z_new)
        output = torch.log_softmax(c, dim=1)
        return output

    def training_step(self, train_batch, batch_idx):
        lab, txt, img, name, test_description = train_batch
        lab = train_batch[lab]
        txt = train_batch[txt]
        img = train_batch[img]
        name = train_batch[name]
        test_description = train_batch[test_description]

        logits, attention_weights = self.image_classifier(img, txt)
        probabilities = torch.softmax(logits, dim=1)
        prediction_probability, predicted_label = torch.max(probabilities, dim=1)
        predicted_label = predicted_label.detach().cpu().numpy()
        prediction_probability = prediction_probability.detach().cpu().numpy()

        loss = F.cross_entropy(logits, lab)
        self.log('train_loss', loss)
        if self.current_epoch == (self.trainer.max_epochs - 1):
           #print('attention_weights',attention_weights)
           t= self.trainer.max_epochs - 1
           for i in range(len(img)):
               image_path = '/DATA/atul_2221cs20/jitendra/MAMI_2022_images/training_images/'+name[i]
                # Assuming name contains the image paths
               #print(image_path)
               text_description=test_description[i]
               process_image(image_path, attention_weights[i],t,text_description)
        return loss

#     def validation_step(self, val_batch, batch_idx):
#         lab, txt, img, name, test_description = val_batch
#         lab = val_batch[lab]
#         txt = val_batch[txt]
#         img = val_batch[img]
#         test_description = val_batch[test_description]

#         logits, attention_weights = self.image_classifier(img, txt)
#         probabilities = torch.softmax(logits, dim=1)
#         prediction_probability, predicted_label = torch.max(probabilities, dim=1)
#         predicted_label = predicted_label.detach().cpu().numpy()
#         prediction_probability = prediction_probability.detach().cpu().numpy()

#         loss = self.cross_entropy_loss(logits, lab)
#         self.log('val_acc', accuracy_score(lab, predicted_label))
#         self.log('val_roc_auc', roc_auc_score(lab, predicted_label))
#         self.log('val_loss', loss)
#         return {
#             'progress_bar': {'val_acc': accuracy_score(lab, predicted_label)},
#             'val_f1_offensive': f1_score(lab, predicted_label, average='macro')
#         }

    def validation_epoch_end(self, validation_step_outputs):
        outs = [out['progress_bar']['val_acc'] for out in validation_step_outputs]
        outs_f1 = [out['val_f1_offensive'] for out in validation_step_outputs]
        self.log('val_acc_all_offn', sum(outs) / len(outs))
        self.log('val_f1_offensive', sum(outs_f1) / len(outs_f1))
        print(f'***val_acc_all_offn at epoch end {sum(outs) / len(outs)}****')
        print(f'***val_f1_offensive at epoch end {sum(outs_f1) / len(outs_f1)}****')

    def test_step(self, batch, batch_idx):
        lab, txt, img, name, text_description = batch
        lab = batch[lab]
        name = batch[name]
        text_description = batch[text_description]
        txt = batch[txt]
        img = batch[img]

        logits, attention_weights = self.image_classifier(img, txt)
        probabilities = torch.softmax(logits, dim=1)
        prediction_probability, predicted_label = torch.max(probabilities, dim=1)
        predicted_label = predicted_label.detach().cpu().numpy()
        prediction_probability = prediction_probability.detach().cpu().numpy()

        loss = F.cross_entropy(logits, lab)
        lab = lab.detach().cpu().numpy()
        self.log('test_acc', accuracy_score(lab, predicted_label))
        self.log('test_roc_auc', roc_auc_score(lab, predicted_label))
        self.log('test_loss', loss)
        tqdm_dict = {'test_acc': accuracy_score(lab, predicted_label)}

        return {
            'progress_bar': tqdm_dict,
            'test_acc': accuracy_score(lab, predicted_label),
            'test_f1_score': f1_score(lab, predicted_label, average='macro'),
        }

    def test_epoch_end(self, outputs):
        outs = [out['test_acc'] for out in outputs]
        outs_f1 = [out['test_f1_score'] for out in outputs]
        self.log('test_acc', sum(outs) / len(outs))
        self.log('test_f1_score', sum(outs_f1) / len(outs_f1))

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-5)
        return optimizer


class HmDataModule(pl.LightningDataModule):
    def setup(self, stage):
        self.hm_train = t_p
        self.hm_val = v_p
        self.hm_test = te_p

    def train_dataloader(self):
        return DataLoader(self.hm_train, batch_size=128, drop_last=True)

#     def val_dataloader(self):
#         return DataLoader(self.hm_val, batch_size=128, drop_last=True)

    def test_dataloader(self):
        return DataLoader(self.hm_test, batch_size=128, drop_last=True)

# Data setup
data_module = HmDataModule()
checkpoint_callback = ModelCheckpoint(
    monitor='val_acc_all_offn',
    dirpath='Jitendra/',
    filename="epoch{epoch:02d}-val_f1_all_offn{val_acc_all_offn:.2f}",
    auto_insert_metric_name=False,
    save_top_k=1,
    mode="max",
)
all_callbacks = [checkpoint_callback]

# Model training
seed_everything(42, workers=True)
hm_model = Classifier()
gpus = 1 if torch.cuda.is_available() else 0
trainer = pl.Trainer(gpus=gpus, deterministic=True, max_epochs=1, precision=16, callbacks=all_callbacks)
trainer.fit(hm_model, data_module)

Global seed set to 42
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name             | Type            | Params
-----------------------------------------------------
0 | MFB              | MFB             | 16.8 M
1 | fin_y_shape      | Linear          | 393 K 
2 | clip_model       | CLIP            | 151 M 
3 | fin_old          | Linear          | 130   
4 | fin              | Linear          | 786 K 
5 | image_classifier | ImageClassifier | 152 M 
-----------------------------------------------------
321 M     Trainable params
0         Non-trainable params
321 M     Total params
643.721   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
#version03
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from transformers import CLIPProcessor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import clip
import cv2
import numpy as np
from PIL import Image
import os
import csv
from pytorch_lightning import seed_everything


# Load CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

# Define attention mechanism
class SelfAttention(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.query_proj = nn.Linear(feature_dim, feature_dim).to(device)
        self.key_proj = nn.Linear(feature_dim, feature_dim).to(device)
        self.value_proj = nn.Linear(feature_dim, feature_dim).to(device)

    def forward(self, image_features):
        #print(' image_features', image_features.size)
        feature_dim = 512

        # Convert NumPy array to PyTorch tensor and move to device
        image_features = torch.tensor(image_features).to(device).float()
       # print("image_features shape (input):", image_features.shape)


        # Apply linear projections for Q, K, and V
        Q = self.query_proj(image_features)
        K = self.key_proj(image_features)
        V = self.value_proj(image_features)

        # Compute attention scores and weights
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / image_features.shape[-1]**0.5
        attention_weights = F.softmax(attention_scores, dim=-1)


        # Apply attention weights to obtain the weighted features
        weighted_features = torch.matmul(attention_weights, V)


        # Sum along the last dimension to get the final attention output
        #output_features = torch.sum(weighted_features, dim=-2)
        print(' weighted_features', weighted_features)
        print("weighted_features shape (output):", weighted_features.shape)

        return weighted_features , attention_weights
# Define PredictionModel
class PredictionModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 512).to(device)
        self.text_embedding = nn.Linear(512, 512).to(device)  # Project text to 512 dimensions
        self.output_layer = nn.Linear(512 + 512, num_classes).to(device)  # Combine 1024 features
    def forward(self, masked_features, text_features=None):
        masked_features = masked_features.float()
        x = F.relu(self.fc1(masked_features))

        if text_features is not None:
            text_features = text_features.float()
            text_emb = F.relu(self.text_embedding(text_features))
           # print("x shape:", x.shape)
           # print("text_emb shape:", text_emb.shape)
            x = torch.cat([x, text_emb], dim=1)

        prediction = self.output_layer(x)
        print('prediction',prediction)
        return prediction

# Define ImageClassifier
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.clip_model = clip_model
        self.attention_layer = SelfAttention(feature_dim=512)
        self.classifier = PredictionModel(input_dim=512, num_classes=2)

    def forward(self, image_tensor, text_tensor):
        #image_tensor = image_tensor.unsqueeze(1)
       # image_tensor = image_tensor.repeat(1, 3, 1, 1)
        #image_features = clip_model.encode_image(image_tensor)
        #print('image_tensor',image_tensor.shape)
        #print('text_tensor',text_tensor.shape)
        masked_features, attention_weights = self.attention_layer(image_tensor)
        print('masked_features',masked_features)
        print('masked_features shape',masked_features.shape)
        prediction = self.classifier(masked_features, text_tensor)
        print('classifier prediction', prediction )
        return prediction, attention_weights

def apply_blur(image, attention_weights, image_path):
    original_image = cv2.imread(image_path)
    attention_weights_np = attention_weights.detach().cpu().numpy()
    threshold = 0.008
    blur_radius = 15

    # Create mask based on attention weights (thresholding)
    blur_mask = attention_weights_np <= threshold  # Corrected the condition

    # Convert blur_mask to uint8 for resizing
    blur_mask = blur_mask.astype('uint8')

    # Resize
    blur_mask = cv2.resize(blur_mask, (original_image.shape[1], original_image.shape[0]), interpolation=cv2.INTER_NEAREST)

    # Apply blur
    blurred_image = cv2.GaussianBlur(original_image.copy(), (blur_radius, blur_radius), 0, borderType=cv2.BORDER_CONSTANT)

    # Ensure the shapes are consistent
    blurred_image = blurred_image[..., ::-1]  # Reverse the order of channels (BGR to RGB)

    # Apply the mask pixel by pixel
    modified_image = np.copy(original_image)
    for i in range(original_image.shape[0]):
        for j in range(original_image.shape[1]):
            if blur_mask[i, j] > 0:
                modified_image[i, j] = blurred_image[i, j]

    return modified_image
def process_image(image_path,attention_weights,t):
    image = cv2.imread(image_path)


    # Optional: You can use the attention weights for visualization
    attention_heatmap = cv2.resize(attention_weights[0].detach().cpu().numpy(), (image.shape[1], image.shape[0]))

    # Apply blur only to misogynistic regions
    modified_image = apply_blur(image, attention_weights,image_path)

    # Save the modified image and attention heatmap
    filename, ext = os.path.splitext(os.path.basename(image_path))
    modified_filename = filename + "_modified" + ext
    attention_heatmap_filename = filename + "_attention_heatmap" + ext

    output_dir = "output_images1"+str(t)
    os.makedirs(output_dir, exist_ok=True)

    modified_path = os.path.join(output_dir, modified_filename)
    heatmap_path = os.path.join(output_dir, attention_heatmap_filename)

    cv2.imwrite(modified_path, modified_image)
    cv2.imwrite(heatmap_path, (attention_heatmap * 255).astype(np.uint8))

    return modified_image, attention_heatmap
# Define Classifier class
class Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.MFB = MFB(512, 512, True, 256, 64, 0.1)
        self.fin_y_shape = torch.nn.Linear(768, 512)
        self.clip_model, self.compose = clip.load("ViT-B/32", device=device)
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
        self.fin_old = torch.nn.Linear(64, 2)  # producing final output logits
        self.fin = torch.nn.Linear(16 * 768, 64)  # linear layer to the fused features
        self.image_classifier = ImageClassifier()

    def mask_and_predict(self, text):
        inputs = clip.tokenize(text, truncate=True).to(self.device)
        text_features = self.clip_model.encode_text(inputs).detach().cpu().numpy()
        text_features = torch.tensor(text_features).to(self.device)
        text_features = text_features.view(text_features.size(0), -1)
        return text_features

    def forward(self, x, y, text):
        x_, y_ = x, y
        text_features = self.mask_and_predict(text)
        z = self.MFB(torch.unsqueeze(y, axis=1), torch.unsqueeze(text_features, axis=1))
        z_new = torch.squeeze(z, dim=1)
        c = self.fin_old(z_new)
        output = torch.log_softmax(c, dim=1)
        return output

    def training_step(self, train_batch, batch_idx):
        lab, txt, img, name, test_description = train_batch
        lab = train_batch[lab]
        txt = train_batch[txt]
        img = train_batch[img]
        name = train_batch[name]
        test_description = train_batch[test_description]

        logits, attention_weights = self.image_classifier(img, txt)
        probabilities = torch.softmax(logits, dim=1)
        prediction_probability, predicted_label = torch.max(probabilities, dim=1)
        predicted_label = predicted_label.detach().cpu().numpy()
        prediction_probability = prediction_probability.detach().cpu().numpy()

        loss = F.cross_entropy(logits, lab)
        self.log('train_loss', loss)
        if self.current_epoch == (self.trainer.max_epochs - 1):
           print('attention_weights',attention_weights)
           t= self.trainer.max_epochs - 1
           for i in range(len(img)):
               image_path = '/DATA/atul_2221cs20/jitendra/MAMI_2022_images/training_images/'+name[i]
                # Assuming name contains the image paths
               #print(image_path)
               process_image(image_path, attention_weights[i],t)
        return loss

#     def validation_step(self, val_batch, batch_idx):
#         lab, txt, img, name, test_description = val_batch
#         lab = val_batch[lab]
#         txt = val_batch[txt]
#         img = val_batch[img]
#         test_description = val_batch[test_description]

#         logits, attention_weights = self.image_classifier(img, txt)
#         probabilities = torch.softmax(logits, dim=1)
#         prediction_probability, predicted_label = torch.max(probabilities, dim=1)
#         predicted_label = predicted_label.detach().cpu().numpy()
#         prediction_probability = prediction_probability.detach().cpu().numpy()

#         loss = self.cross_entropy_loss(logits, lab)
#         self.log('val_acc', accuracy_score(lab, predicted_label))
#         self.log('val_roc_auc', roc_auc_score(lab, predicted_label))
#         self.log('val_loss', loss)
#         return {
#             'progress_bar': {'val_acc': accuracy_score(lab, predicted_label)},
#             'val_f1_offensive': f1_score(lab, predicted_label, average='macro')
#         }

    def validation_epoch_end(self, validation_step_outputs):
        outs = [out['progress_bar']['val_acc'] for out in validation_step_outputs]
        outs_f1 = [out['val_f1_offensive'] for out in validation_step_outputs]
        self.log('val_acc_all_offn', sum(outs) / len(outs))
        self.log('val_f1_offensive', sum(outs_f1) / len(outs_f1))
        print(f'***val_acc_all_offn at epoch end {sum(outs) / len(outs)}****')
        print(f'***val_f1_offensive at epoch end {sum(outs_f1) / len(outs_f1)}****')

    def test_step(self, batch, batch_idx):
        lab, txt, img, name, text_description = batch
        lab = batch[lab]
        name = batch[name]
        text_description = batch[text_description]
        txt = batch[txt]
        img = batch[img]

        logits, attention_weights = self.image_classifier(img, txt)
        probabilities = torch.softmax(logits, dim=1)
        prediction_probability, predicted_label = torch.max(probabilities, dim=1)
        predicted_label = predicted_label.detach().cpu().numpy()
        prediction_probability = prediction_probability.detach().cpu().numpy()

        loss = F.cross_entropy(logits, lab)
        lab = lab.detach().cpu().numpy()
        self.log('test_acc', accuracy_score(lab, predicted_label))
        self.log('test_roc_auc', roc_auc_score(lab, predicted_label))
        self.log('test_loss', loss)
        tqdm_dict = {'test_acc': accuracy_score(lab, predicted_label)}

        return {
            'progress_bar': tqdm_dict,
            'test_acc': accuracy_score(lab, predicted_label),
            'test_f1_score': f1_score(lab, predicted_label, average='macro'),
        }

    def test_epoch_end(self, outputs):
        outs = [out['test_acc'] for out in outputs]
        outs_f1 = [out['test_f1_score'] for out in outputs]
        self.log('test_acc', sum(outs) / len(outs))
        self.log('test_f1_score', sum(outs_f1) / len(outs_f1))

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-5)
        return optimizer


class HmDataModule(pl.LightningDataModule):
    def setup(self, stage):
        self.hm_train = t_p
        self.hm_val = v_p
        self.hm_test = te_p

    def train_dataloader(self):
        return DataLoader(self.hm_train, batch_size=128, drop_last=True)

#     def val_dataloader(self):
#         return DataLoader(self.hm_val, batch_size=128, drop_last=True)

    def test_dataloader(self):
        return DataLoader(self.hm_test, batch_size=128, drop_last=True)

# Data setup
data_module = HmDataModule()
checkpoint_callback = ModelCheckpoint(
    monitor='val_acc_all_offn',
    dirpath='Jitendra/',
    filename="epoch{epoch:02d}-val_f1_all_offn{val_acc_all_offn:.2f}",
    auto_insert_metric_name=False,
    save_top_k=1,
    mode="max",
)
all_callbacks = [checkpoint_callback]

# Model training
seed_everything(42, workers=True)
hm_model = Classifier()
gpus = 1 if torch.cuda.is_available() else 0
trainer = pl.Trainer(gpus=gpus, deterministic=True, max_epochs=1, precision=16, callbacks=all_callbacks)
trainer.fit(hm_model, data_module)

Global seed set to 42
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name             | Type            | Params
-----------------------------------------------------
0 | MFB              | MFB             | 16.8 M
1 | fin_y_shape      | Linear          | 393 K 
2 | clip_model       | CLIP            | 151 M 
3 | fin_old          | Linear          | 130   
4 | fin              | Linear          | 786 K 
5 | image_classifier | ImageClassifier | 152 M 
-----------------------------------------------------
321 M     Trainable params
0         Non-trainable params
321 M     Total params
643.721   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


 weighted_features tensor([[ 0.0130,  0.2676, -0.1196,  ..., -0.1937, -0.0708,  0.0917],
        [ 0.0131,  0.2683, -0.1176,  ..., -0.1920, -0.0693,  0.0922],
        [ 0.0139,  0.2676, -0.1172,  ..., -0.1931, -0.0695,  0.0917],
        ...,
        [ 0.0142,  0.2668, -0.1169,  ..., -0.1930, -0.0675,  0.0919],
        [ 0.0128,  0.2676, -0.1168,  ..., -0.1926, -0.0692,  0.0920],
        [ 0.0128,  0.2676, -0.1188,  ..., -0.1924, -0.0695,  0.0918]],
       device='cuda:0', dtype=torch.float16, grad_fn=<MmBackward0>)
weighted_features shape (output): torch.Size([128, 512])
masked_features tensor([[ 0.0130,  0.2676, -0.1196,  ..., -0.1937, -0.0708,  0.0917],
        [ 0.0131,  0.2683, -0.1176,  ..., -0.1920, -0.0693,  0.0922],
        [ 0.0139,  0.2676, -0.1172,  ..., -0.1931, -0.0695,  0.0917],
        ...,
        [ 0.0142,  0.2668, -0.1169,  ..., -0.1930, -0.0675,  0.0919],
        [ 0.0128,  0.2676, -0.1168,  ..., -0.1926, -0.0692,  0.0920],
        [ 0.0128,  0.2676, -0.1188,  ..., -

In [ ]:
#just for testing3

class Classifier(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.MFB = MFB(512, 512, True, 256, 64, 0.1)
    self.fin_y_shape = torch.nn.Linear(768, 512)
    self.clip_model, self.compose = clip.load("ViT-B/32", device=device)
    self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
    self.fin_old = torch.nn.Linear(64, 2)  # producing final output logits
    self.fin = torch.nn.Linear(16 * 768, 64)  # linear layer to the fused features

  def mask_and_predict(self, text):
    #print("inside mask text",text)
    inputs = clip.tokenize(text, truncate=True).to(self.device)
   # print("inside mask inputs",inputs)

    #for i in range(1, len(inputs[0]) - 1):
    # masked_input_ids = inputs.clone()
    # masked_input_ids[0][i] = 0  # Assuming 0 is the index for the [MASK] token
    text_features = self.clip_model.encode_text(inputs).detach().cpu().numpy()
    #print("inside the mask text_features1",text_features,text_features.shape)
    text_features = torch.tensor(text_features).to(self.device)
    #print("inside the mask text_features2",text_features,text_features.shape)
    text_features = text_features.view(text_features.size(0), -1)

    return text_features

  def forward(self, x, y, text):

    x_, y_ = x, y
    text_features = self.mask_and_predict(text)
    z = self.MFB(torch.unsqueeze(y, axis=1), torch.unsqueeze(text_features, axis=1))
    z_new = torch.squeeze(z, dim=1)
    c = self.fin_old(z_new)
    output = torch.log_softmax(c, dim=1)
    return output
  def forward1(self, x, y):

    x_, y_ = x, y
    #text_features = self.mask_and_predict(text)
    z = self.MFB(torch.unsqueeze(y, axis=1), torch.unsqueeze(x, axis=1))
    z_new = torch.squeeze(z, dim=1)
    c = self.fin_old(z_new)
    output = torch.log_softmax(c, dim=1)
    return output

  def cross_entropy_loss(self, logits, labels):

    return F.nll_loss(logits, labels)
  def training_step(self, train_batch, batch_idx):

    lab, txt, img, name, test_description = train_batch
    lab = train_batch[lab]
    txt = train_batch[txt]
    img = train_batch[img]
    name = train_batch[name]
    test_description = train_batch[test_description]
    logits = self.forward1(txt, img)
    probabilities = torch.softmax(logits, dim=1)
    #print( probabilities)
    prediction_probability, predicted_label = torch.max(probabilities, dim=1)
    predicted_label = predicted_label.detach().cpu().numpy()
    prediction_probability = prediction_probability.detach().cpu().numpy()

    csv_file_path = f"hateful_words_results_epoch_{self.current_epoch + 1}.csv"
    with open(csv_file_path, mode='a', newline='') as file:
        fieldnames = ["image_name", "actual_label", "predicted_label", "prediction_probability",
                      "top_hateful_word_1", "confidence_1", "top_hateful_word_2", "confidence_2",
                      "top_hateful_word_3", "confidence_3",
                      "top_hateful_word_4", "confidence_4",
                      "top_hateful_word_5", "confidence_5",
                      "top_hateful_word_6", "confidence_6",
                      "top_hateful_word_7", "confidence_7",
                      "top_hateful_word_8", "confidence_8"
                     ]
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Write header only if the file is empty
        if file.tell() == 0:
            writer.writeheader()

        for img_name, text_description, label,predicted_label,prediction_probability in zip(name, test_description, lab,predicted_label,prediction_probability):
            predictions = []
            word_impact = []
            predicted_label=predicted_label.item()
            prediction_probability=prediction_probability.item()
            text_description_words = text_description.split()

            # Iterate over each text description in the list
            for description in text_description_words:
                masked_text = ' '.join([word for word in text_description_words if word != description])
                word_input = description
                word_logits = self.forward(txt, img, masked_text)
               # print("word_logits",word_logits)
                word_prediction = F.softmax(word_logits, dim=1)
                #print("word_prediction",word_prediction)
                hate_confidence = word_prediction[0][1].item()
                #print("hate_confidence",hate_confidence)
                word_impact.append((description, hate_confidence))

            # Sort the words based on hate confidence in descending order
            sorted_word_impact = sorted(word_impact, key=lambda x: x[1], )

            # Take the top 3 most hateful words if available, otherwise fill with placeholders
            top_hateful_words = sorted_word_impact[:8] if sorted_word_impact else [("", 0)] * 8

            # Get predicted label and probability for the current image


            # Append the results for this image to the CSV file
            writer.writerow({
                "image_name": img_name,
                "actual_label": label.item(),
                "predicted_label": predicted_label,
                "prediction_probability": prediction_probability,
                "top_hateful_word_1": top_hateful_words[0][0],
                "confidence_1": top_hateful_words[0][1],
                "top_hateful_word_2": top_hateful_words[1][0] if len(top_hateful_words) > 1 else "",
                "confidence_2": top_hateful_words[1][1] if len(top_hateful_words) > 1 else 0,
                "top_hateful_word_3": top_hateful_words[2][0] if len(top_hateful_words) > 2 else "",
                "confidence_3": top_hateful_words[2][1] if len(top_hateful_words) > 2 else 0,
                "top_hateful_word_4": top_hateful_words[3][0] if len(top_hateful_words) > 3 else "",
                "confidence_4": top_hateful_words[3][1] if len(top_hateful_words) > 3 else 0,
                "top_hateful_word_5": top_hateful_words[4][0] if len(top_hateful_words) > 4 else "",
                "confidence_5": top_hateful_words[4][1] if len(top_hateful_words) > 4 else 0,
                "top_hateful_word_6": top_hateful_words[5][0] if len(top_hateful_words) > 5 else "",
                "confidence_6": top_hateful_words[5][1] if len(top_hateful_words) > 5 else 0,
                "top_hateful_word_7": top_hateful_words[6][0] if len(top_hateful_words) > 6 else "",
                "confidence_7": top_hateful_words[6][1] if len(top_hateful_words) > 6 else 0,
                "top_hateful_word_8": top_hateful_words[7][0] if len(top_hateful_words) > 7 else "",
                "confidence_8": top_hateful_words[7][1] if len(top_hateful_words) > 7 else 0,
            })

    loss = self.cross_entropy_loss(logits, lab)
    self.log('train_loss', loss)
    return loss


  def validation_step(self, val_batch, batch_idx):
     # print(self)
      lab,txt,img,name,test_description= val_batch
      #print(lab)
      lab = val_batch[lab]
      #print(lab)
      txt = val_batch[txt]
      img = val_batch[img]
      test_description=val_batch[test_description]


      logits= self.forward1(txt,img)
      probabilities = torch.softmax(logits, dim=1)
      prediction_probability, predicted_label = torch.max(probabilities, dim=1)
      predicted_label = predicted_label.detach().cpu().numpy()
      prediction_probability = prediction_probability.detach().cpu().numpy()
      tmp = predicted_label
      #print(np.argmax(logits.detach().cpu().numpy(),axis=-1))
      loss = self.cross_entropy_loss(logits, lab)
      #print(loss)
      lab = lab.detach().cpu().numpy()
      for img_name, text_description, label,predicted_label,prediction_probability in zip(name, test_description, lab,predicted_label,prediction_probability):
            predictions = []
            word_impact = []
            predicted_label=predicted_label.item()
            prediction_probability=prediction_probability.item()
            text_description_words = text_description.split()

            # Iterate over each text description in the list
            for description in text_description_words:
                masked_text = ' '.join([word for word in text_description_words if word != description])
                word_input = description
                word_logits = self.forward(txt, img, masked_text)
               # print("word_logits",word_logits)
                word_prediction = F.softmax(word_logits, dim=1)
                #print("word_prediction",word_prediction)
                hate_confidence = word_prediction[0][1].item()
                #print("hate_confidence",hate_confidence)
                word_impact.append((description, hate_confidence))

            # Sort the words based on hate confidence in descending order
            sorted_word_impact = sorted(word_impact, key=lambda x: x[1], )

            # Take the top 3 most hateful words if available, otherwise fill with placeholders
            top_hateful_words = sorted_word_impact[:8] if sorted_word_impact else [("", 0)] * 8

#*****************************************

      self.log('val_acc', accuracy_score(lab,tmp))
      #print('val_acc', accuracy_score(lab,tmp))
      self.log('val_roc_auc',roc_auc_score(lab,tmp))
      #print('val_roc_auc',roc_auc_score(lab,tmp))
      self.log('val_loss', loss)
      #print('val_loss', loss)
      tqdm_dict = {'val_acc': accuracy_score(lab,tmp)}
      #print('val_acc', accuracy_score(lab,tmp))
      return {
                'progress_bar': tqdm_dict,
      'val_f1 offensive': f1_score(lab,tmp,average='macro')
      }

  def validation_epoch_end(self, validation_step_outputs):
    #print(validation_step_outputs)
    outs = []
    outs1,outs2,outs3,outs4,outs5,outs6,outs7,outs8,outs9,outs10,outs11,outs12,outs13,outs14,outs16,outs17 = \
    [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
    for out in validation_step_outputs:
      #print(out)
      outs.append(out['progress_bar']['val_acc'])
      outs14.append(out['val_f1 offensive'])

    self.log('val_acc_all_offn', sum(outs)/len(outs))
    self.log('val_f1 offensive', sum(outs14)/len(outs14))
    print(f'***val_acc_all_offn at epoch end {sum(outs)/len(outs)}****')
    print(f'***val_f1 offensive at epoch end {sum(outs14)/len(outs14)}****')


  def test_step(self, batch, batch_idx):

        lab,txt,img,name,text_description= batch

        lab = batch[lab]
        name=batch[name]
        test_description=batch[text_description]
        txt = batch[txt]
        img = batch[img]
        logits= self.forward1(txt,img)

        probabilities = torch.softmax(logits, dim=1)

        prediction_probability, predicted_label = torch.max(probabilities, dim=1)
        predicted_label = predicted_label.detach().cpu().numpy()
        tmp=predicted_label
        prediction_probability = prediction_probability.detach().cpu().numpy()

        csv_file_path = f"hateful_words_test_results_epoch__{self.current_epoch + 1}.csv"
        with open(csv_file_path, mode='a', newline='') as file:
            fieldnames = ["image_name", "actual_label", "predicted_label", "prediction_probability",
                          "top_hateful_word_1", "confidence_1", "top_hateful_word_2", "confidence_2",
                          "top_hateful_word_3", "confidence_3",
                          "top_hateful_word_4", "confidence_4",
                          "top_hateful_word_5", "confidence_5",
                          "top_hateful_word_6", "confidence_6",
                          "top_hateful_word_7", "confidence_7",
                          "top_hateful_word_8", "confidence_8"
                         ]
            writer = csv.DictWriter(file, fieldnames=fieldnames)

            # Write header only if the file is empty
            if file.tell() == 0:
                writer.writeheader()

            for img_name, text_description, label,predicted_label,prediction_probability in zip(name, test_description, lab,predicted_label,prediction_probability):
                predictions = []
                word_impact = []
                predicted_label=predicted_label.item()
                prediction_probability=prediction_probability.item()
                text_description_words = text_description.split()

                # Iterate over each text description in the list
                for description in text_description_words:
                    masked_text = ' '.join([word for word in text_description_words if word != description])
                    word_input = description
                    word_logits = self.forward(txt, img, masked_text)
                   # print("word_logits",word_logits)
                    word_prediction = F.softmax(word_logits, dim=1)
                    #print("word_prediction",word_prediction)
                    hate_confidence = word_prediction[0][1].item()
                    #print("hate_confidence",hate_confidence)
                    word_impact.append((description, hate_confidence))

                # Sort the words based on hate confidence in descending order
                sorted_word_impact = sorted(word_impact, key=lambda x: x[1], )

                # Take the top 3 most hateful words if available, otherwise fill with placeholders
                top_hateful_words = sorted_word_impact[:8] if sorted_word_impact else [("", 0)] * 8

                # Get predicted label and probability for the current image


                # Append the results for this image to the CSV file
                writer.writerow({
                    "image_name": img_name,
                    "actual_label": label.item(),
                    "predicted_label": predicted_label,
                    "prediction_probability": prediction_probability,
                    "top_hateful_word_1": top_hateful_words[0][0],
                    "confidence_1": top_hateful_words[0][1],
                    "top_hateful_word_2": top_hateful_words[1][0] if len(top_hateful_words) > 1 else "",
                    "confidence_2": top_hateful_words[1][1] if len(top_hateful_words) > 1 else 0,
                    "top_hateful_word_3": top_hateful_words[2][0] if len(top_hateful_words) > 2 else "",
                    "confidence_3": top_hateful_words[2][1] if len(top_hateful_words) > 2 else 0,
                    "top_hateful_word_4": top_hateful_words[3][0] if len(top_hateful_words) > 3 else "",
                    "confidence_4": top_hateful_words[3][1] if len(top_hateful_words) > 3 else 0,
                    "top_hateful_word_5": top_hateful_words[4][0] if len(top_hateful_words) > 4 else "",
                    "confidence_5": top_hateful_words[4][1] if len(top_hateful_words) > 4 else 0,
                    "top_hateful_word_6": top_hateful_words[5][0] if len(top_hateful_words) > 5 else "",
                    "confidence_6": top_hateful_words[5][1] if len(top_hateful_words) > 5 else 0,
                    "top_hateful_word_7": top_hateful_words[6][0] if len(top_hateful_words) > 6 else "",
                    "confidence_7": top_hateful_words[6][1] if len(top_hateful_words) > 6 else 0,
                    "top_hateful_word_8": top_hateful_words[7][0] if len(top_hateful_words) > 7 else "",
                    "confidence_8": top_hateful_words[7][1] if len(top_hateful_words) > 7 else 0,
                })
        #****************************************************************
        loss = self.cross_entropy_loss(logits, lab)
        lab = lab.detach().cpu().numpy()
        self.log('test_acc', accuracy_score(lab,tmp))
        self.log('test_roc_auc',roc_auc_score(lab,tmp))
        self.log('test_loss', loss)
        tqdm_dict = {'test_acc': accuracy_score(lab,tmp)}
        #labels = ["hate" if pred == 1 else "not hate" for pred in tmp]

        return {
                'progress_bar': tqdm_dict,
                'test_acc': accuracy_score(lab,tmp),
                'test_f1_score': f1_score(lab,tmp,average='macro'),
                                  #predicted label

        }


  def test_epoch_end(self, outputs):
      # OPTIONAL
      outs = []
      outs1,outs2,outs3,outs4,outs5,outs6,outs7,outs8,outs9,outs10,outs11,outs12,outs13,outs14 = \
      [],[],[],[],[],[],[],[],[],[],[],[],[],[]
      for out in outputs:
        outs.append(out['test_acc'])
        outs2.append(out['test_f1_score'])
      self.log('test_acc', sum(outs)/len(outs))
      self.log('test_f1_score', sum(outs2)/len(outs2))


  def configure_optimizers(self):

    # optimizer = torch.optim.Adam(self.parameters(), lr=3e-2)
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-5)
    #print(torch.optim.Adam(self.parameters(), lr=1e-5))

    return optimizer


class HmDataModule(pl.LightningDataModule):


  def setup(self, stage):

    self.hm_train = t_p

    self.hm_val = v_p

    # self.hm_test = test
    self.hm_test = te_p


  def train_dataloader(self):
    return DataLoader(self.hm_train, batch_size=128, drop_last=True)

  def val_dataloader(self):
    return DataLoader(self.hm_val, batch_size=128, drop_last=True)

  def test_dataloader(self):
    return DataLoader(self.hm_test, batch_size=128, drop_last=True)
data_module = HmDataModule()
#print(data_module)
checkpoint_callback = ModelCheckpoint(
     monitor='val_acc_all_offn',
     dirpath='Jitendra/',
     filename= "epoch{epoch:02d}-val_f1_all_offn{val_acc_all_offn:.2f}",#'epoch{epoch:02d}-val_f1_all_offn{val_acc_all_offn:.2f}',
     auto_insert_metric_name=False,
     save_top_k=1,
    mode="max",
 )
all_callbacks = []
all_callbacks.append(checkpoint_callback)
from pytorch_lightning import seed_everything
seed_everything(42, workers=True)
hm_model = Classifier()
gpus = 1 if torch.cuda.is_available() else 0
trainer = pl.Trainer(gpus=gpus,deterministic=True,max_epochs=1,precision=16,callbacks=all_callbacks)
trainer.fit(hm_model, data_module)

Global seed set to 42
/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /DATA/atul_2221cs20/jitendra/Jitendra exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]

  | Name        | Type   | Params
------------------------

Sanity Checking: 0it [00:00, ?it/s]

/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


***val_acc_all_offn at epoch end 0.54296875****
***val_f1 offensive at epoch end 0.38690682942646515****


/DATA/atul_2221cs20/anaconda3/envs/jitu/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

In [ ]:
test_dataloader = DataLoader(dataset=te_p, batch_size=1478)

ckpt_path = '/DATA/atul_2221cs20/jitendra/Jitendra/epoch00-val_f1_all_offn0.79.ckpt' # put ckpt_path according to the path output in the previous cell
#ckpt_path = "/DATA/atul_2221cs20/jitendra/Jitendra/Current.ckpt"
trainer.test(dataloaders=test_dataloader,ckpt_path=ckpt_path)

Restoring states from the checkpoint path at /DATA/atul_2221cs20/jitendra/Jitendra/epoch00-val_f1_all_offn0.79.ckpt


RuntimeError: Error(s) in loading state_dict for Classifier:
	Missing key(s) in state_dict: "clip_model.positional_embedding", "clip_model.text_projection", "clip_model.logit_scale", "clip_model.visual.class_embedding", "clip_model.visual.positional_embedding", "clip_model.visual.proj", "clip_model.visual.conv1.weight", "clip_model.visual.ln_pre.weight", "clip_model.visual.ln_pre.bias", "clip_model.visual.transformer.resblocks.0.attn.in_proj_weight", "clip_model.visual.transformer.resblocks.0.attn.in_proj_bias", "clip_model.visual.transformer.resblocks.0.attn.out_proj.weight", "clip_model.visual.transformer.resblocks.0.attn.out_proj.bias", "clip_model.visual.transformer.resblocks.0.ln_1.weight", "clip_model.visual.transformer.resblocks.0.ln_1.bias", "clip_model.visual.transformer.resblocks.0.mlp.c_fc.weight", "clip_model.visual.transformer.resblocks.0.mlp.c_fc.bias", "clip_model.visual.transformer.resblocks.0.mlp.c_proj.weight", "clip_model.visual.transformer.resblocks.0.mlp.c_proj.bia